In [ ]:
import Pkg; Pkg.add("DifferentialEquations")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("Formatting")
import Pkg; Pkg.add("Catalyst")
import Pkg; Pkg.add("Noise")
import Pkg; Pkg.add("LinearAlgebra")
import Pkg; Pkg.add("Latexify")
import Pkg; Pkg.add("ModelingToolkit")
import Pkg; Pkg.add("PlotlyJS")
import Pkg; Pkg.add("MATLABDiffEq")

_ = IJulia.clear_output(true)

In [ ]:
using DifferentialEquations;
using Random;
using Plots;
using Formatting;
using LinearAlgebra;
using Noise;
using Catalyst;
using Latexify;
using MATLABDiffEq;

# Utils

In [ ]:
# Returns the sum of concentrations of the species with a given prefix
function aggregate_values(rn, odesol; prefix='x')
    ss = species(rn)

    ret = zeros(length(odesol[1, :]))
    total = 0
    for i in 1:length(ss)
        item = ss[i]
        item_str = "$item"
        if startswith(item_str, prefix)
#             println(item_str)
            ret += odesol[i, :] # Gets the variable at index i for all time steps.
        end
    end
    return ret
end


function print_species_array(reaction_network)
    ss = species(reaction_network)
    print("species = [")
    for s in ss
        print("'", s,"'", ",")
    end
    print("]")
    
end


function simulate(rn, u0, p; tspan=(0.0, 1000.0), reltol=1e-12, abstol=1e-12)
    oprob = ODEProblem(rn, u0, tspan, p)
    sol = solve(oprob, Rosenbrock23(), reltol=reltol, abstol=abstol)
    return sol
end

In [ ]:
unimol_autocat = @reaction_network uni begin
    k, a + G_i --> i + waste1
    k, i + G_a_a --> a + a + o + waste2
    k, o + Fl_f --> f + waste3
end k
species(unimol_autocat)

In [ ]:
u0 = [
   1e-9, 1e-8, 0.0, 0.0, 1e-8, 0.0, 0.0, 1e-7, 0.0, 0.0
]
p = [1.5e6]
tspan = (0, 3600.0)

oprob = ODEProblem(unimol_autocat, u0, tspan, p)
sol = solve(oprob, AutoTsit5(Rosenbrock23()), reltol=1e-12, abstol=1e-12)

In [ ]:
plot()
a = aggregate_values(unimol_autocat, sol, prefix="a")
i = aggregate_values(unimol_autocat, sol, prefix="i")
f = aggregate_values(unimol_autocat, sol, prefix="f")
plot!(sol.t, [a i f], label=["a(t)"  "i(t)" "f(t)"], legend=:topleft)
savefig("polymeraseCRN.png")